In [161]:
import requests as rq
from bs4 import BeautifulSoup


url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2024.06.12


In [162]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

20240612


In [163]:
import requests as rq 
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_opt_stk = {
    'mktId': 'STK',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false', 
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referrer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url , gen_opt_stk, headers=headers).text

print(otp_stk)

axOjD65vykZjFzMlDTqqG+TwsHuNg1Pc4zN4mQJ3zRYRtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eR3aYTk0jdw2SEZxARBjVo4tBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [164]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code' : otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head() #코스피 종목의 산업별 현황 데이터가 저장되었음 

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4740,25,0.53,214498077660
1,006840,AK홀딩스,KOSPI,기타금융,14780,0,0.00,195798951580
2,027410,BGF,KOSPI,기타금융,3595,20,0.56,344101863645
3,282330,BGF리테일,KOSPI,유통업,115900,700,0.61,2003204705400
4,138930,BNK금융지주,KOSPI,기타금융,8120,40,0.50,2615358116560


In [165]:
gen_opt_ksq = {
    'mktId': 'KSQ',
    'trdDd': biz_day,
    'money': '1',
    'csvxls_isNo': 'false', 
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url , gen_opt_ksq, headers=headers).text
down_sector_ksq = rq.post(down_url, {'code' : otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding='EUC-KR')

sector_ksq.head() #코스닥 종목의 산업별 현황 데이터가 저장되었음 

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2830,75,2.72,144569536860
1,054620,APS,KOSDAQ,금융,6800,160,2.41,135280702800
2,265520,AP시스템,KOSDAQ,반도체,28600,600,2.14,437048640600
3,211270,AP위성,KOSDAQ,통신장비,15700,-100,-0.63,236792172800
4,109960,AP헬스케어,KOSDAQ,유통,758,11,1.47,86527425208


In [166]:
krx_sector = pd.concat([sector_stk,sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4740,25,0.53,214498077660,20240612
1,006840,AK홀딩스,KOSPI,기타금융,14780,0,0.00,195798951580,20240612
2,027410,BGF,KOSPI,기타금융,3595,20,0.56,344101863645,20240612
3,282330,BGF리테일,KOSPI,유통업,115900,700,0.61,2003204705400,20240612
4,138930,BNK금융지주,KOSPI,기타금융,8120,40,0.50,2615358116560,20240612


In [167]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_data = {
    'searchType': '1',
    'mktId': 'ALL',
    'trdDd': biz_day,
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp = rq.post(gen_otp_url, gen_otp_data, headers=headers).text

down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
krx_ind = rq.post(down_url, {'code': otp}, headers=headers)

krx_ind = pd.read_csv(BytesIO(krx_ind.content), encoding='EUC-KR')
krx_ind['종목명'] = krx_ind['종목명'].str.strip()
krx_ind['기준일'] = biz_day

krx_ind.head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률,기준일
0,060310,3S,2830,75,2.72,30.0,94.33,NaN,NaN,947.0,2.99,0,0.00,20240612
1,095570,AJ네트웍스,4740,25,0.53,367.0,12.92,873.0,5.43,9326.0,0.51,270,5.70,20240612
2,006840,AK홀딩스,14780,0,0.00,2635.0,5.61,NaN,NaN,44339.0,0.33,200,1.35,20240612
3,054620,APS,6800,160,2.41,667.0,10.19,NaN,NaN,11683.0,0.58,0,0.00,20240612
4,265520,AP시스템,28600,600,2.14,3997.0,7.16,4581.0,6.24,21396.0,1.34,270,0.94,20240612


In [168]:
diff = list(set(krx_sector['종목명']).symmetric_difference(set(krx_ind['종목명'])))
print(diff)

['골든센츄리', '크리스탈신소재', '프레스티지바이오파마', '롯데리츠', '한화리츠', '오가닉티코스메틱', '애머릿지', '소마젠', 'GRT', '고스트스튜디오', '씨엑스아이', 'JTC', '미래에셋맵스리츠', '코람코더원리츠', '네오이뮨텍', '마스턴프리미어리츠', '코오롱티슈진', '디앤디플랫폼리츠', 'NH올원리츠', '맵스리얼티1', '엘브이엠씨홀딩스', '한국ANKOR유전', 'SK리츠', '이지스밸류리츠', 'KB스타리츠', 'NH프라임리츠', '헝셩그룹', '이리츠코크렙', '미래에셋글로벌리츠', 'ESR켄달스퀘어리츠', '한국패러랠', '컬러레이', '코람코라이프인프라리츠', '엑세스바이오', '잉글우드랩', '제이알글로벌리츠', '로스웰', '이지스레지던스리츠', '글로벌에스엠', '신한알파리츠', '이스트아시아홀딩스', '신한서부티엔디리츠', '스타리츠', '삼성FN리츠', 'SBI핀테크솔루션즈', '윙입푸드', '케이탑리츠', '맥쿼리인프라', '에이리츠']


In [169]:
kor_ticker = pd.merge(krx_sector,
                      krx_ind,
                      on=krx_sector.columns.intersection(
                          krx_ind.columns).tolist(),
                      how='outer')

kor_ticker.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,KOSPI,서비스업,4740,25,0.53,214498077660,20240612,367.0,12.92,873.0,5.43,9326.0,0.51,270.0,5.70
1,006840,AK홀딩스,KOSPI,기타금융,14780,0,0.00,195798951580,20240612,2635.0,5.61,NaN,NaN,44339.0,0.33,200.0,1.35
2,027410,BGF,KOSPI,기타금융,3595,20,0.56,344101863645,20240612,813.0,4.42,658.0,5.46,17286.0,0.21,120.0,3.34
3,282330,BGF리테일,KOSPI,유통업,115900,700,0.61,2003204705400,20240612,11337.0,10.22,12126.0,9.56,62197.0,1.86,4100.0,3.54
4,138930,BNK금융지주,KOSPI,기타금융,8120,40,0.50,2615358116560,20240612,1905.0,4.26,2573.0,3.16,31746.0,0.26,510.0,6.28


In [170]:
import numpy as np

kor_ticker['종목구분'] = np.where(kor_ticker['종목명'].str.contains('스팩|제[0-9]+호'), '스팩', 
                              np.where(kor_ticker['종목코드'].str[-1:] != '0', '우선주',
                                       np.where(kor_ticker['종목명'].str.endswith('리츠'), '리츠',
                                                np.where(kor_ticker['종목명'].isin(diff),  '기타',
                                                '보통주'))))
kor_ticker = kor_ticker.reset_index(drop=True)
kor_ticker.columns = kor_ticker.columns.str.replace(' ', '')
kor_ticker = kor_ticker[['종목코드', '종목명', '시장구분', '종가',
                         '시가총액', '기준일', 'EPS', '선행EPS', 'BPS', '주당배당금', '종목구분']]
kor_ticker = kor_ticker.replace({np.nan: None})
kor_ticker['기준일'] = pd.to_datetime(kor_ticker['기준일'])

kor_ticker.head()

# 스펙 (SPAC): 특정 기업의 인수나 합병을 통해 상장시키기 위해 만들어진 특수 목적 인수 회사. (페이퍼컴퍼니를 상장하고 , 그를 통해 모은 돈으로 다른 맘에 드는 회사를 인수 ,합병하기 위함)
# 우선주 (Preferred Stock): 배당금과 자산 청구권에서 우선권을 가지며, 일반적으로 의결권이 없는 주식.
# 리츠 (REITs): 부동산에 투자하여 발생하는 수익을 투자자에게 배당하는 투자회사로, 높은 배당 수익을 제공.

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4740,214498077660,2024-06-12,367.0,873.0,9326.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,14780,195798951580,2024-06-12,2635.0,None,44339.0,200.0,보통주
2,027410,BGF,KOSPI,3595,344101863645,2024-06-12,813.0,658.0,17286.0,120.0,보통주
3,282330,BGF리테일,KOSPI,115900,2003204705400,2024-06-12,11337.0,12126.0,62197.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,8120,2615358116560,2024-06-12,1905.0,2573.0,31746.0,510.0,보통주


In [171]:
import pymysql 

con = pymysql.connect(
user= 'root',
passwd = 'Skdmlekdrh0707/',
host = '127.0.0.1',
db = 'stock_db',
charset='utf8'
)

In [172]:
mycursor = con.cursor()
query = f"""
    insert into kor_ticker (종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분)
    values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    종목명=new.종목명,시장구분=new.시장구분,종가=new.종가,시가총액=new.시가총액,EPS=new.EPS,선행EPS=new.선행EPS,
    BPS=new.BPS,주당배당금=new.주당배당금,종목구분 = new.종목구분;
"""

args = kor_ticker.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

In [173]:
kor_ticker

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,095570,AJ네트웍스,KOSPI,4740,214498077660,2024-06-12,367.0,873.0,9326.0,270.0,보통주
1,006840,AK홀딩스,KOSPI,14780,195798951580,2024-06-12,2635.0,None,44339.0,200.0,보통주
2,027410,BGF,KOSPI,3595,344101863645,2024-06-12,813.0,658.0,17286.0,120.0,보통주
3,282330,BGF리테일,KOSPI,115900,2003204705400,2024-06-12,11337.0,12126.0,62197.0,4100.0,보통주
4,138930,BNK금융지주,KOSPI,8120,2615358116560,2024-06-12,1905.0,2573.0,31746.0,510.0,보통주
...,...,...,...,...,...,...,...,...,...,...,...
2679,024060,흥구석유,KOSDAQ,15120,226800000000,2024-06-12,78.0,None,5440.0,50.0,보통주
2680,010240,흥국,KOSDAQ,5450,67158693200,2024-06-12,1182.0,None,8598.0,240.0,보통주
2681,189980,흥국에프엔비,KOSDAQ,2435,97735608745,2024-06-12,221.0,None,2449.0,40.0,보통주
2682,037440,희림,KOSDAQ,6380,88825390500,2024-06-12,489.0,1113.0,5583.0,150.0,보통주


In [174]:
import json
import requests as rq
import pandas as pd

url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd=G10'''
data = rq.get(url).json()

type(data)

dict

In [175]:
print(data.keys())

dict_keys(['info', 'list', 'sector', 'size'])


In [176]:
data_pd = pd.json_normalize(data['list'])

data_pd.head()

,IDX_CD,IDX_NM_KOR,ALL_MKT_VAL,CMP_CD,CMP_KOR,MKT_VAL,WGT,S_WGT,CAL_WGT,SEC_CD,SEC_NM_KOR,SEQ,TOP60,APT_SHR_CNT
0,G10,WICS 에너지,20882510,096770,SK이노베이션,5586363,26.75,26.75,1.0,G10,에너지,1,4,53611930
1,G10,WICS 에너지,20882510,009830,한화솔루션,3671109,17.58,44.33,1.0,G10,에너지,2,4,108292298
2,G10,WICS 에너지,20882510,267250,HD현대,3069987,14.70,59.03,1.0,G10,에너지,3,4,44236128
3,G10,WICS 에너지,20882510,010950,S-Oil,2790927,13.36,72.40,1.0,G10,에너지,4,4,41655633
4,G10,WICS 에너지,20882510,078930,GS,2132315,10.21,82.61,1.0,G10,에너지,5,4,49245150


In [177]:
import time
import json
import requests as rq
import pandas as pd
from tqdm import tqdm

sector_code = [
    'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45'
]

data_sector = []

for i in tqdm(sector_code):
    url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''    
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])

    data_sector.append(data_pd)

    time.sleep(2)

kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])

100%|██████████| 10/10 [00:22<00:00,  2.22s/it]


In [178]:
import pymysql

con = pymysql.connect(user='root',
                      passwd='Skdmlekdrh0707/',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')

mycursor = con.cursor()
query = f"""
    insert into kor_sector (IDX_CD, CMP_CD, CMP_KOR, SEC_NM_KOR, 기준일)
    values (%s,%s,%s,%s,%s) as new
    on duplicate key update
    IDX_CD = new.IDX_CD, CMP_KOR = new.CMP_KOR, SEC_NM_KOR = new.SEC_NM_KOR
"""

args = kor_sector.values.tolist()

mycursor.executemany(query, args)
con.commit()

con.close()

# 전 종목 정보 가져오기 

In [179]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('mysql+pymysql://root:Skdmlekdrh0707/@127.0.0.1:3306/stock_db')
query = """
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query, con=engine)
engine.dispose()

ticker_list.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분
0,000020,동화약품,KOSPI,8140.0,2.273620e+11,2024-06-12,991.0,NaN,13413.0,180.0,보통주
1,000040,KR모터스,KOSPI,643.0,3.866540e+10,2024-06-12,NaN,NaN,618.0,0.0,보통주
2,000050,경방,KOSPI,7970.0,2.185000e+11,2024-06-12,NaN,NaN,29623.0,125.0,보통주
3,000070,삼양홀딩스,KOSPI,70400.0,6.029250e+11,2024-06-12,22269.0,NaN,257475.0,3500.0,보통주
4,000080,하이트진로,KOSPI,20750.0,1.455270e+12,2024-06-12,512.0,1429.0,15694.0,950.0,보통주


In [180]:
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from datetime import date

i = 0
ticker = ticker_list['종목코드'][i]
fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d")
to = (date.today()).strftime("%Y%m%d")

url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
&startTime={fr}&endTime={to}&timeframe=day'''

data = rq.get(url).content
data_price = pd.read_csv(BytesIO(data))

data_price.head()

,[['날짜','시가','고가','저가','종가','거래량','외국인소진율'],Unnamed: 7
0,"[""20190614""",9680.0,9870.0,9680.0,9730.0,56319.0,6.79],NaN
1,"[""20190617""",9790.0,9830.0,9650.0,9790.0,41302.0,6.77],NaN
2,"[""20190618""",9790.0,10150.0,9680.0,9900.0,152876.0,6.65],NaN
3,"[""20190619""",9900.0,9960.0,9730.0,9900.0,66758.0,6.64],NaN
4,"[""20190620""",9930.0,9950.0,9810.0,9820.0,20025.0,6.63],NaN


In [181]:
import re

price = data_price.iloc[:, 0:6]
price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
price = price.dropna()
price['날짜'] = price['날짜'].str.extract('(\d+)')
price['날짜'] = pd.to_datetime(price['날짜'])
price['종목코드'] = ticker

price.head()

,날짜,시가,고가,저가,종가,거래량,종목코드
0,2019-06-14,9680.0,9870.0,9680.0,9730.0,56319.0,000020
1,2019-06-17,9790.0,9830.0,9650.0,9790.0,41302.0,000020
2,2019-06-18,9790.0,10150.0,9680.0,9900.0,152876.0,000020
3,2019-06-19,9900.0,9960.0,9730.0,9900.0,66758.0,000020
4,2019-06-20,9930.0,9950.0,9810.0,9820.0,20025.0,000020


In [182]:
# 패키지 불러오기
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta
import requests as rq
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# DB 연결
engine = create_engine('mysql+pymysql://root:Skdmlekdrh0707/@127.0.0.1:3306/stock_db')
con = pymysql.connect(user='root',
                      passwd='Skdmlekdrh0707/',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')
mycursor = con.cursor()

# 티커리스트 불러오기
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
""", con=engine)

# DB 저장 쿼리
query = """
    insert into kor_price (날짜, 시가, 고가, 저가, 종가, 거래량, 종목코드)
    values (%s,%s,%s,%s,%s,%s,%s) as new
    on duplicate key update
    시가 = new.시가, 고가 = new.고가, 저가 = new.저가,
    종가 = new.종가, 거래량 = new.거래량;
"""

# 오류 발생시 저장할 리스트 생성
error_list = []

# 티커 처리 함수
def process_ticker(ticker):
    try:
        # 시작일과 종료일
        fr = (date.today() + relativedelta(years=-5)).strftime("%Y%m%d")
        to = (date.today()).strftime("%Y%m%d")

        # url 생성
        url = f'''https://fchart.stock.naver.com/siseJson.nhn?symbol={ticker}&requestType=1
        &startTime={fr}&endTime={to}&timeframe=day'''

        # 데이터 다운로드
        for _ in range(3):  # 최대 3번 재시도
            try:
                data = rq.get(url).content
                break
            except rq.RequestException:
                time.sleep(1)
        else:
            return None, ticker

        # 데이터 클렌징
        data_price = pd.read_csv(BytesIO(data))
        price = data_price.iloc[:, 0:6]
        price.columns = ['날짜', '시가', '고가', '저가', '종가', '거래량']
        price = price.dropna()
        price['날짜'] = price['날짜'].str.extract('(\d+)')
        price['날짜'] = pd.to_datetime(price['날짜'])
        price['종목코드'] = ticker

        # 주가 데이터를 DB에 저장
        return price.values.tolist(), None

    except Exception as e:
        return None, ticker

batch_size = 100
args_list = []
error_list = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_ticker, ticker) for ticker in ticker_list['종목코드']]
    for future in tqdm(as_completed(futures), total=len(futures)):
        args, error = future.result()
        if args:
            args_list.extend(args)
            if len(args_list) >= batch_size:
                mycursor.executemany(query, args_list)
                con.commit()
                args_list = []
        if error:
            error_list.append(error)

# 마지막 남은 데이터 처리
if args_list:
    mycursor.executemany(query, args_list)
    con.commit()

# DB 연결 종료
engine.dispose()
con.close()

100%|██████████| 2431/2431 [02:33<00:00, 15.85it/s] 


# 재무제표 추가해보기 

In [183]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('mysql+pymysql://root:Skdmlekdrh0707/@127.0.0.1:3306/stock_db')
query = """
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
"""
ticker_list = pd.read_sql(query, con=engine)
engine.dispose()

i = 0
ticker = ticker_list['종목코드'][i]

url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
data = pd.read_html(url, displayed_only=False)

[item.head(3) for item in data]

[  IFRS(연결)  2021/12  2022/12  2023/12  2024/03   전년동기 전년동기(%)
 0      매출액   2930.0   3404.0   3611.0   1189.0  994.0    19.6
 1     매출원가   1437.0   1594.0   1707.0    639.0  468.0    36.6
 2    매출총이익   1493.0   1810.0   1904.0    549.0  526.0     4.4,
   IFRS(연결)  2023/06  2023/09  2023/12  2024/03   전년동기 전년동기(%)
 0      매출액    900.0    875.0    842.0   1189.0  994.0    19.6
 1     매출원가    438.0    424.0    377.0    639.0  468.0    36.6
 2    매출총이익    462.0    450.0    466.0    549.0  526.0     4.4,
              IFRS(연결)  2021/12  2022/12  2023/12  2024/03
 0                  자산   4478.0   4611.0   5650.0   5581.0
 1  유동자산계산에 참여한 계정 펼치기   2202.0   2275.0   2377.0   2337.0
 2                재고자산    362.0    468.0    707.0    704.0,
              IFRS(연결)  2023/06  2023/09  2023/12  2024/03
 0                  자산   4818.0   4902.0   5650.0   5581.0
 1  유동자산계산에 참여한 계정 펼치기   2317.0   2346.0   2377.0   2337.0
 2                재고자산    549.0    547.0    707.0    704.0,
          IFRS(연결)  

In [184]:
print(data[0].columns.tolist(), '\n',
      data[2].columns.tolist(), '\n',
      data[4].columns.tolist()
     )

['IFRS(연결)', '2021/12', '2022/12', '2023/12', '2024/03', '전년동기', '전년동기(%)'] 
 ['IFRS(연결)', '2021/12', '2022/12', '2023/12', '2024/03'] 
 ['IFRS(연결)', '2021/12', '2022/12', '2023/12', '2024/03']


In [185]:
data_fs_y = pd.concat(
    [data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2], data[4]])
data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})

data_fs_y.head()

,계정,2021/12,2022/12,2023/12,2024/03
0,매출액,2930.0,3404.0,3611.0,1189.0
1,매출원가,1437.0,1594.0,1707.0,639.0
2,매출총이익,1493.0,1810.0,1904.0,549.0
3,판매비와관리비계산에 참여한 계정 펼치기,1269.0,1511.0,1716.0,484.0
4,인건비,468.0,489.0,521.0,153.0


In [186]:
import requests as rq
from bs4 import BeautifulSoup
import re

page_data = rq.get(url)
page_data_html = BeautifulSoup(page_data.content, 'html.parser')

fiscal_data = page_data_html.select('div.corp_group1 > h2')
fiscal_data_text = fiscal_data[1].text
fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)

print(fiscal_data_text)

['12']


In [187]:
data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') |
                          (data_fs_y.columns.str[-2:].isin(fiscal_data_text))]
data_fs_y.head()

,계정,2021/12,2022/12,2023/12
0,매출액,2930.0,3404.0,3611.0
1,매출원가,1437.0,1594.0,1707.0
2,매출총이익,1493.0,1810.0,1904.0
3,판매비와관리비계산에 참여한 계정 펼치기,1269.0,1511.0,1716.0
4,인건비,468.0,489.0,521.0


In [188]:
data_fs_y[data_fs_y.loc[:, ~data_fs_y.columns.isin(['계정'])].isna().all(
    axis=1)].head()

,계정,2021/12,2022/12,2023/12
10,기타원가성비용,NaN,NaN,NaN
18,대손충당금환입액,NaN,NaN,NaN
19,매출채권처분이익,NaN,NaN,NaN
20,당기손익-공정가치측정 금융자산관련이익,NaN,NaN,NaN
23,금융자산손상차손환입,NaN,NaN,NaN


In [189]:
data_fs_y['계정'].value_counts(ascending=False).head()

계정
기타          4
배당금수익       3
파생상품이익      3
이자수익        3
법인세납부(-)    3
Name: count, dtype: int64

In [190]:
def clean_fs(df, ticker, frequency):
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)

    # 날짜 형식 변환
    try:
        df['기준일'] = pd.to_datetime(df['기준일'], format='%Y-%m') + pd.tseries.offsets.MonthEnd()
    except ValueError:
        df['기준일'] = pd.to_datetime(df['기준일'], format='%Y/%m') + pd.tseries.offsets.MonthEnd()

    df['종목코드'] = ticker
    df['공시구분'] = frequency

    return df

In [191]:
data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')

data_fs_y_clean.head()

,계정,기준일,값,종목코드,공시구분
0,매출액,2021-12-31,2930.0,000020,y
1,매출원가,2021-12-31,1437.0,000020,y
2,매출총이익,2021-12-31,1493.0,000020,y
3,판매비와관리비,2021-12-31,1269.0,000020,y
4,인건비,2021-12-31,468.0,000020,y


In [192]:
# 분기 데이터

data_fs_q = pd.concat(
    [data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3], data[5]])
data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})
data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

data_fs_q_clean.head()

,계정,기준일,값,종목코드,공시구분
0,매출액,2023-06-30,900.0,000020,q
1,매출원가,2023-06-30,438.0,000020,q
2,매출총이익,2023-06-30,462.0,000020,q
3,판매비와관리비,2023-06-30,407.0,000020,q
4,인건비,2023-06-30,129.0,000020,q


In [193]:
# 패키지 불러오기
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# DB 연결
engine = create_engine('mysql+pymysql://root:Skdmlekdrh0707/@127.0.0.1:3306/stock_db')
con = pymysql.connect(user='root',
                      passwd='Skdmlekdrh0707/',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')
mycursor = con.cursor()

# 티커리스트 불러오기
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
    and 종목구분 = '보통주';
""", con=engine)

# DB 저장 쿼리
query = """
    insert into kor_fs (계정, 기준일, 값, 종목코드, 공시구분)
    values (%s,%s,%s,%s,%s) as new
    on duplicate key update
    값=new.값
"""

# 오류 발생시 저장할 리스트 생성
error_list = []

# 재무제표 클렌징 함수
def clean_fs(df, ticker, frequency):
    df = df[~df.loc[:, ~df.columns.isin(['계정'])].isna().all(axis=1)]
    df = df.drop_duplicates(['계정'], keep='first')
    df = pd.melt(df, id_vars='계정', var_name='기준일', value_name='값')
    df = df[~pd.isnull(df['값'])]
    df['계정'] = df['계정'].replace({'계산에 참여한 계정 펼치기': ''}, regex=True)

    # 날짜 형식 변환
    try:
        df['기준일'] = pd.to_datetime(df['기준일'], format='%Y-%m') + pd.tseries.offsets.MonthEnd()
    except ValueError:
        df['기준일'] = pd.to_datetime(df['기준일'], format='%Y/%m') + pd.tseries.offsets.MonthEnd()

    df['종목코드'] = ticker
    df['공시구분'] = frequency

    return df

# 티커 처리 함수
def process_ticker(ticker):
    try:
        url = f'http://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&gicode=A{ticker}'
        data = pd.read_html(url, displayed_only=False)

        data_fs_y = pd.concat([
            data[0].iloc[:, ~data[0].columns.str.contains('전년동기')], data[2],
            data[4]
        ])
        data_fs_y = data_fs_y.rename(columns={data_fs_y.columns[0]: "계정"})

        page_data = rq.get(url)
        page_data_html = BeautifulSoup(page_data.content, 'html.parser')
        fiscal_data = page_data_html.select('div.corp_group1 > h2')
        fiscal_data_text = fiscal_data[1].text
        fiscal_data_text = re.findall('[0-9]+', fiscal_data_text)

        data_fs_y = data_fs_y.loc[:, (data_fs_y.columns == '계정') | (
            data_fs_y.columns.str[-2:].isin(fiscal_data_text))]

        data_fs_y_clean = clean_fs(data_fs_y, ticker, 'y')

        data_fs_q = pd.concat([
            data[1].iloc[:, ~data[1].columns.str.contains('전년동기')], data[3],
            data[5]
        ])
        data_fs_q = data_fs_q.rename(columns={data_fs_q.columns[0]: "계정"})

        data_fs_q_clean = clean_fs(data_fs_q, ticker, 'q')

        data_fs_bind = pd.concat([data_fs_y_clean, data_fs_q_clean])
        return data_fs_bind.values.tolist(), None

    except Exception as e:
        return None, ticker

batch_size = 100
args_list = []
error_list = []

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(process_ticker, ticker) for ticker in ticker_list['종목코드']]
    for future in tqdm(as_completed(futures), total=len(futures)):
        args, error = future.result()
        if args:
            args_list.extend(args)
            if len(args_list) >= batch_size:
                mycursor.executemany(query, args_list)
                con.commit()
                args_list = []
        if error:
            error_list.append(error)

# 마지막 남은 데이터 처리
if args_list:
    mycursor.executemany(query, args_list)
    con.commit()

# DB 연결 종료
engine.dispose()
con.close()

100%|██████████| 2431/2431 [06:45<00:00,  5.99it/s]


# 가치지표 계산

In [194]:
# 패키지 불러오기
from sqlalchemy import create_engine
import pandas as pd

# DB 연결
engine = create_engine('mysql+pymysql://root:Skdmlekdrh0707/@127.0.0.1:3306/stock_db')

# 티커 리스트
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
	and 종목구분 = '보통주';
""", con=engine)

# 삼성전자 분기 재무제표
sample_fs = pd.read_sql("""
select * from kor_fs
where 공시구분 = 'q'
and 종목코드 = '005930'
and 계정 in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
""", con=engine)

engine.dispose()

In [195]:
sample_fs = sample_fs.sort_values(['종목코드', '계정', '기준일'])

sample_fs.head()

,계정,기준일,값,종목코드,공시구분
0,당기순이익,2023-06-30,17236.0,005930,q
1,당기순이익,2023-09-30,58442.0,005930,q
2,당기순이익,2023-12-31,63448.0,005930,q
3,당기순이익,2024-03-31,67547.0,005930,q
4,매출액,2023-06-30,600055.0,005930,q


In [196]:
sample_fs['ttm'] = sample_fs.groupby(
    ['종목코드', '계정'], as_index=False)['값'].rolling(window=4,
                                                 min_periods=4).sum()['값']

sample_fs

,계정,기준일,값,종목코드,공시구분,ttm
0,당기순이익,2023-06-30,17236.0,005930,q,NaN
1,당기순이익,2023-09-30,58442.0,005930,q,NaN
2,당기순이익,2023-12-31,63448.0,005930,q,NaN
3,당기순이익,2024-03-31,67547.0,005930,q,206673.0
4,매출액,2023-06-30,600055.0,005930,q,NaN
5,매출액,2023-09-30,674047.0,005930,q,NaN
6,매출액,2023-12-31,677799.0,005930,q,NaN
7,매출액,2024-03-31,719156.0,005930,q,2671057.0
8,영업활동으로인한현금흐름,2023-06-30,81699.0,005930,q,NaN
9,영업활동으로인한현금흐름,2023-09-30,97305.0,005930,q,NaN


In [197]:
sample_fs['ttm'] = sample_fs.groupby(
    ['종목코드', '계정'], as_index=False)['값'].rolling(window=4,
                                                 min_periods=4).sum()['값']

sample_fs

,계정,기준일,값,종목코드,공시구분,ttm
0,당기순이익,2023-06-30,17236.0,005930,q,NaN
1,당기순이익,2023-09-30,58442.0,005930,q,NaN
2,당기순이익,2023-12-31,63448.0,005930,q,NaN
3,당기순이익,2024-03-31,67547.0,005930,q,206673.0
4,매출액,2023-06-30,600055.0,005930,q,NaN
5,매출액,2023-09-30,674047.0,005930,q,NaN
6,매출액,2023-12-31,677799.0,005930,q,NaN
7,매출액,2024-03-31,719156.0,005930,q,2671057.0
8,영업활동으로인한현금흐름,2023-06-30,81699.0,005930,q,NaN
9,영업활동으로인한현금흐름,2023-09-30,97305.0,005930,q,NaN


In [198]:
import numpy as np

sample_fs['ttm'] = np.where(sample_fs['계정'] == '자본',
                            sample_fs['ttm'] / 4, sample_fs['ttm'])
sample_fs = sample_fs.groupby(['계정', '종목코드']).tail(1)

sample_fs.head()

,계정,기준일,값,종목코드,공시구분,ttm
3,당기순이익,2024-03-31,67547.0,005930,q,206673.0
7,매출액,2024-03-31,719156.0,005930,q,2671057.0
11,영업활동으로인한현금흐름,2024-03-31,118663.0,005930,q,497119.0
15,자본,2024-03-31,3719160.0,005930,q,3646157.5


In [199]:
sample_fs_merge = sample_fs[['계정', '종목코드', 'ttm']].merge(
    ticker_list[['종목코드', '시가총액', '기준일']], on='종목코드')
sample_fs_merge['시가총액'] = sample_fs_merge['시가총액']/100000000

sample_fs_merge.head()

,계정,종목코드,ttm,시가총액,기준일
0,당기순이익,005930,206673.0,4566880.0,2024-06-12
1,매출액,005930,2671057.0,4566880.0,2024-06-12
2,영업활동으로인한현금흐름,005930,497119.0,4566880.0,2024-06-12
3,자본,005930,3646157.5,4566880.0,2024-06-12


In [200]:
sample_fs_merge['value'] = sample_fs_merge['시가총액'] / sample_fs_merge['ttm']
sample_fs_merge['지표'] = np.where(
    sample_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        sample_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(sample_fs_merge['계정'] == '자본', 'PBR',
                 np.where(sample_fs_merge['계정'] == '당기순이익', 'PER', None))))

sample_fs_merge

,계정,종목코드,ttm,시가총액,기준일,value,지표
0,당기순이익,005930,206673.0,4566880.0,2024-06-12,22.097129,PER
1,매출액,005930,2671057.0,4566880.0,2024-06-12,1.709765,PSR
2,영업활동으로인한현금흐름,005930,497119.0,4566880.0,2024-06-12,9.186694,PCR
3,자본,005930,3646157.5,4566880.0,2024-06-12,1.252519,PBR


In [201]:
ticker_list_sample = ticker_list[ticker_list['종목코드'] == '005930'].copy()
ticker_list_sample['DY'] = ticker_list_sample['주당배당금'] / ticker_list_sample['종가']

ticker_list_sample.head()

,종목코드,종목명,시장구분,종가,시가총액,기준일,EPS,선행EPS,BPS,주당배당금,종목구분,DY
258,005930,삼성전자,KOSPI,76500.0,4.566880e+14,2024-06-12,2131.0,5908.0,52002.0,1444.0,보통주,0.018876


In [202]:
# 패키지 불러오기
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

# DB 연결
engine = create_engine('mysql+pymysql://root:Skdmlekdrh0707/@127.0.0.1:3306/stock_db')
con = pymysql.connect(user='root',
                      passwd='Skdmlekdrh0707/',
                      host='127.0.0.1',
                      db='stock_db',
                      charset='utf8')
mycursor = con.cursor()

# 분기 재무제표 불러오기
kor_fs = pd.read_sql("""
select * from kor_fs
where 공시구분 = 'q'
and 계정 in ('당기순이익', '자본', '영업활동으로인한현금흐름', '매출액');
""", con=engine)

# 티커 리스트 불러오기
ticker_list = pd.read_sql("""
select * from kor_ticker
where 기준일 = (select max(기준일) from kor_ticker) 
and 종목구분 = '보통주';
""", con=engine)

engine.dispose()

In [203]:
# TTM 구하기
kor_fs = kor_fs.sort_values(['종목코드', '계정', '기준일'])
kor_fs['ttm'] = kor_fs.groupby(['종목코드', '계정'], as_index=False)['값'].rolling(
    window=4, min_periods=4).sum()['값']

# 자본은 평균 구하기
kor_fs['ttm'] = np.where(kor_fs['계정'] == '자본', kor_fs['ttm'] / 4,
                         kor_fs['ttm'])
kor_fs = kor_fs.groupby(['계정', '종목코드']).tail(1)

In [204]:
kor_fs_merge = kor_fs[['계정', '종목코드',
                       'ttm']].merge(ticker_list[['종목코드', '시가총액', '기준일']],
                                     on='종목코드')
kor_fs_merge['시가총액'] = kor_fs_merge['시가총액'] / 100000000

kor_fs_merge['value'] = kor_fs_merge['시가총액'] / kor_fs_merge['ttm']
kor_fs_merge['value'] = kor_fs_merge['value'].round(4)
kor_fs_merge['지표'] = np.where(
    kor_fs_merge['계정'] == '매출액', 'PSR',
    np.where(
        kor_fs_merge['계정'] == '영업활동으로인한현금흐름', 'PCR',
        np.where(kor_fs_merge['계정'] == '자본', 'PBR',
                 np.where(kor_fs_merge['계정'] == '당기순이익', 'PER', None))))

kor_fs_merge.rename(columns={'value': '값'}, inplace=True)
kor_fs_merge = kor_fs_merge[['종목코드', '기준일', '지표', '값']]
kor_fs_merge = kor_fs_merge.replace([np.inf, -np.inf, np.nan], None)

kor_fs_merge.head(4)

,종목코드,기준일,지표,값
0,000020,2024-06-12,PER,12.4242
1,000020,2024-06-12,PSR,0.5974
2,000020,2024-06-12,PCR,7.4545
3,000020,2024-06-12,PBR,0.5658


In [205]:
query = """
    insert into kor_value (종목코드, 기준일, 지표, 값)
    values (%s,%s,%s,%s) as new
    on duplicate key update
    값=new.값
"""

args_fs = kor_fs_merge.values.tolist()
mycursor.executemany(query, args_fs)
con.commit()

In [206]:
ticker_list['값'] = ticker_list['주당배당금'] / ticker_list['종가']
ticker_list['값'] = ticker_list['값'].round(4)
ticker_list['지표'] = 'DY'
dy_list = ticker_list[['종목코드', '기준일', '지표', '값']]
dy_list = dy_list.replace([np.inf, -np.inf, np.nan], None)
dy_list = dy_list[dy_list['값'] != 0]

dy_list.head()

,종목코드,기준일,지표,값
0,000020,2024-06-12,DY,0.0221
2,000050,2024-06-12,DY,0.0157
3,000070,2024-06-12,DY,0.0497
4,000080,2024-06-12,DY,0.0458
5,000100,2024-06-12,DY,0.0061


In [207]:
args_dy = dy_list.values.tolist()
mycursor.executemany(query, args_dy)
con.commit()

engine.dispose()
con.close()